In [1]:
import pandas
import os
import json
import numpy as np
df = pandas.read_json('./data/logs.json')


In [22]:
df2 = df.copy()
df2['i'] = np.arange(len(df2))
df3 = df2.copy()
df2 = df2.explode('sentences')
df2['signalWord'] = df2['sentences'].map(lambda x: [[x['words'][i], 0 if i in x['unknownIndice'] else 1]for i in range(len(x['words']))])
df3['signalWord'] = df2.groupby('i')['signalWord'].apply(sum)
del df3['i']

In [23]:
df3

,classId,timeZ,eltime,scorePerc,userId,bookId,sentences,chapterId,time,signalWord
0,test,1.000000,121800,-0.867770,SutG8ZQfC0HYGsbcZahj,knkqgp57wJ8C,"[{'unknownWords': [], 'sid': '3ed4b2ee-dc06-11...",3ed570ee-dc06-11e9-8569-48bf6beb4102,2019-09-24T4:46:38+09:00,"[[We, 1], [have, 1], [him, 1], [in, 1], [the, ..."
1,19-grade-8,0.454545,1300,-2.934042,OQJ8K3jcLOM3eYaImkyN,knkqgp57wJ8C,"[{'unknownWords': [], 'sid': '3ed3e184-dc06-11...",3ed570ee-dc06-11e9-8569-48bf6beb4102,2019-09-25T5:3:01+09:00,"[[His, 1], [feelings, 1], [were, 1], [too, 1],..."
2,19-grade-6,0.266667,60000,0.420139,0bHdAlU0BUXj4OLRPzTZ,DZf40OcrwD8C,"[{'unknownWords': [], 'sid': 'b4daef16-dc52-11...",b4db68f6-dc52-11e9-8b48-48bf6beb4102,2019-09-29T7:17:37+09:00,"[[Then, 1], [we, 1], [d, 1], [really, 1], [get..."
3,19-grade-7-joy,0.500000,30800,0.476856,zdWiLjoIxRDrtMRNjOlk,RfSsXCi9FxoC,"[{'unknownWords': [], 'sid': '882f212c-dc51-11...",88302ed2-dc51-11e9-8285-48bf6beb4102,2019-10-06T09:58:48.512Z,"[[Father, 1], [would, 1], [stand, 1], [at, 1],..."
4,19-grade-6,1.000000,31300,0.503747,ZYTj0ZyUTpLsaxb0zoSB,DZf40OcrwD8C,"[{'unknownWords': [], 'sid': 'b4e06fea-dc52-11...",b4e28910-dc52-11e9-8b48-48bf6beb4102,2019-10-14T10:19:25.910Z,"[[There, 1], [they, 1], [visited, 1], [the, 1]..."
...,...,...,...,...,...,...,...,...,...,...
2453,19-grade-6,0.833333,66500,-1.362777,qtW5EhH8cUORCIwwOSiX,DZf40OcrwD8C,"[{'unknownWords': ['assembled'], 'sid': 'b4de7...",b4df1e2e-dc52-11e9-8b48-48bf6beb4102,2019-10-10T9:24:09+09:00,"[[When, 1], [the, 1], [groom, 1], [s, 1], [fam..."
2454,19-grade-6,0.833333,16800,-0.491493,xP0QeRM7qCAi1bOSAt1o,DZf40OcrwD8C,"[{'unknownWords': [], 'sid': 'b4de0dae-dc52-11...",b4df1e2e-dc52-11e9-8b48-48bf6beb4102,2019-10-12T09:54:29.824Z,"[[The, 1], [groom, 1], [s, 1], [parents, 1], [..."
2455,19-grade-6,0.714286,567700,-1.543713,AWJ3AzY0AdAUKDqerSwH,DZf40OcrwD8C,"[{'unknownWords': ['homemade'], 'sid': 'b4e52e...",b4e6e64a-dc52-11e9-8b48-48bf6beb4102,2019-10-19T7:46:07+09:00,"[[She, 1], [saw, 1], [a, 1], [crowd, 1], [of, ..."
2456,19-grade-6,1.000000,140200,0.525148,g11JTV6am9fleET7bIid,DZf40OcrwD8C,"[{'unknownWords': [], 'sid': 'b4e5a4d8-dc52-11...",b4e6e64a-dc52-11e9-8b48-48bf6beb4102,2019-10-22T09:49:00.141Z,"[[And, 1], [I, 1], [ve, 1], [met, 1], [lots, 1..."


In [30]:
df3 = df3.loc[~df3['classId'].isin(['test', 'test2'])]
df3['len'] = len(df3['signalWord'])
userDf = df3[['userId', 'classId', 'eltime', 'scorePerc', 'len']].groupby(['userId', 'classId']).agg('mean')
userDf['eltime'] /= 1000
whiteUserDf = userDf.loc[userDf['eltime'] > 11].reset_index()
df3 = df3.merge(whiteUserDf[['userId']], on='userId', how='inner')

In [31]:
whiteUserDf

,userId,classId,eltime,scorePerc,len
0,0bHdAlU0BUXj4OLRPzTZ,19-grade-6,63.706667,0.454369,1936.0
1,1yz9QcMjyGFMMdC6LPlJ,19-grade-6,53.804545,0.033136,1936.0
2,3dkoVNdFAoch4sxXEtWo,19-grade-6,26.308000,-0.705699,1936.0
3,7ksTSrU6NjGGoZqnu7uG,19-grade-6,111.967742,0.180287,1936.0
4,AWJ3AzY0AdAUKDqerSwH,19-grade-6,28.882249,-0.821458,1936.0
5,GDQqFBeAkT3BM8LaUf4e,19-grade-7-joy,35.259441,-0.449977,1936.0
6,Jj9P1sIqjLLF0wVqDFeX,19-grade-6,57.708163,-0.159752,1936.0
7,NdlYszQsqy4y8pKYKj8N,19-grade-7-joy,89.949123,-1.780588,1936.0
8,OQJ8K3jcLOM3eYaImkyN,19-grade-8,87.883065,-1.682042,1936.0
9,VbFRYXiXVyBLWzGDlqqm,19-grade-6,22.784884,-0.299999,1936.0


In [32]:
whiteUserDf = whiteUserDf.merge(df3.groupby('userId')['userId'].agg('count').to_frame('c'), on='userId', how='inner')

In [33]:
whiteUserDf

,userId,classId,eltime,scorePerc,len,c
0,0bHdAlU0BUXj4OLRPzTZ,19-grade-6,63.706667,0.454369,1936.0,30
1,1yz9QcMjyGFMMdC6LPlJ,19-grade-6,53.804545,0.033136,1936.0,22
2,3dkoVNdFAoch4sxXEtWo,19-grade-6,26.308000,-0.705699,1936.0,100
3,7ksTSrU6NjGGoZqnu7uG,19-grade-6,111.967742,0.180287,1936.0,31
4,AWJ3AzY0AdAUKDqerSwH,19-grade-6,28.882249,-0.821458,1936.0,169
5,GDQqFBeAkT3BM8LaUf4e,19-grade-7-joy,35.259441,-0.449977,1936.0,143
6,Jj9P1sIqjLLF0wVqDFeX,19-grade-6,57.708163,-0.159752,1936.0,147
7,NdlYszQsqy4y8pKYKj8N,19-grade-7-joy,89.949123,-1.780588,1936.0,57
8,OQJ8K3jcLOM3eYaImkyN,19-grade-8,87.883065,-1.682042,1936.0,124
9,VbFRYXiXVyBLWzGDlqqm,19-grade-6,22.784884,-0.299999,1936.0,86


In [59]:
df3[['time', 'userId', 'eltime', 'scorePerc', 'classId', 'signalWord']].to_csv('logs.csv', index=False)